### Transport Tutorial - Notebook 1

# Solve Dantzig's Transport Problem using ixmp4 and linopy

<img style="float: right; height: 80px;" src="_static/python.png">

## Aim and scope of this tutorial

This tutorial takes you through the steps to solve a simple optimization model
using the **ixmp4** database management package and the **linopy** optimization package.

We use **Dantzig's transport problem**, which is used as a [tutorial for linopy](https://linopy.readthedocs.io/en/latest/transport-tutorial.html).
This problem solves for a least-cost shipping schedule that meets demand constraints markets
and supply constraints at factories.

For reference of the transport problem, see:
> Dantzig, G B, Chapter 3.3. In Linear Programming and Extensions.  
> Princeton University Press, Princeton, New Jersey, 1963.

## Tutorial outline

This tutorial consists of three Jupyter notebooks

0. Set up an **ixmp4.Platform** to store the scenario input data and solution
1. Implement the **baseline version of the transport problem** and solve it
2. Create an **alternative scenario** and solve it 

<div class="alert alert-info">

This notebook requires that you set up a database and defined the units as shown in **Notebook 0**.

</div>

## Launching the `Platform` and initializing a new `Run`

A `Platform` is the connection to the database that holds all data and relevant additional information.

A `Run` is an object that holds all relevant information for one quantification of a scenario.  
A run is identified by a model name, a scenario name and a version number (assigned automatically).

You have to *register a new local database* before you can run the tutorial. 

Run the following in the command-line:
```
ixmp4 platforms add tutorial-test
```

You can then check if the database was successfully created by running
```
ixmp4 platforms list
```

After creating the database, you can connect to it via an `ixmp4.Platform` instance.

In [ ]:
import ixmp4

platform = ixmp4.Platform("transport-tutorial")

Now, we initialize a new `ixmp4.Run` in the database.

In [ ]:
run = platform.runs.create(model="transport problem", scenario="standard")

## Defining the `IndexSet`s

An `IndexSet` defines a list of elements with a name. These `IndexSet`s can be used for "indexed assignment" of parameters, variables and equations. 
The entries of these parameters, etc. are then validated against the elements of the linked `IndexSet`(s). 
In database terms, a column of a parameter can be "foreign-keyed" onto an `IndexSet`.

Below, we first show the data as they would be written in the GAMS tutorial ([transport.gms](transport.gms) in this folder).  

We now initialize these sets and assign the elements.

In [ ]:
i = run.optimization.indexsets.create("i")
i.add(["seattle", "san-diego"])

We can display the elements of any `IndexSet` as a Python list:

In [ ]:
i.elements

`IndexSet`s can be notated with documentation strings to record their meaning. These strings can then be used by linopy, too! 

In [ ]:
# TODO We would need to implement this first, though, probably setting pd.Series.name
# to it.

In [ ]:
i.docs = "Canning Plants"

For simplicity, the steps of creating an `IndexSet` and assigning elements can be done in one line.

In [ ]:
run.optimization.indexsets.create("j").add(["new-york", "chicago", "topeka"])

## Assigning the `Parameter`s

Next, we define the parameters *capacity* and *demand*. The parameters are assigned on the indexsets *i* and *j*, respectively.

In [ ]:
import pandas as pd

from ixmp4.core import Unit

# We only need to (and can!) define units once for each Platform
try:
    cases = platform.units.get("cases")
except Unit.NotFound:
    cases = platform.units.create("cases")

# Capacity of plant i in cases
# Parameter data can be a dict...
a = run.optimization.parameters.create(name="a", constrained_to_indexsets=["i"])
a_data = {
    "i": ["seattle", "san-diego"],
    "values": [350, 600],
    "units": [cases.name, cases.name],
}
a.add(data=a_data)

# Demand at market j in cases
b = run.optimization.parameters.create("b", constrained_to_indexsets=["j"])
# ... or a pd.DataFrame
b_data = pd.DataFrame(
    [
        ["new-york", 325, cases.name],
        ["chicago", 300, cases.name],
        ["topeka", 275, cases.name],
    ],
    columns=["j", "values", "units"],
)
b.add(b_data)

Notice how the `parameter.data` has three columns but has only been linked to one `IndexSet`? That's on purpose: Every `Parameter` needs to have (the columns) *values* and *units*, but these cannot be constrained to an `IndexSet`. The value(s) can be any number(s), but the units need to be known to the `Platform`.

Here's how to access `parameter.data` to e.g. quickly confirm that *b* is set correctly:

In [ ]:
b.data

In [ ]:
try:
    km = platform.units.get("km")
except Unit.NotFound:
    km = platform.units.create("km")

# Distance in thousands of miles
d = run.optimization.parameters.create("d", constrained_to_indexsets=["i", "j"])
# You can start with some data ...
d_data = {
    "i": ["seattle", "seattle", "seattle", "san-diego"],
    "j": ["new-york", "chicago", "topeka", "new-york"],
    "values": [2.5, 1.7, 1.8, 2.5],
    "units": [km.name] * 4,
}
d.add(d_data)

# ... and expand it later on:
d.add({"i": ["san-diego"], "j": ["chicago"], "values": [1.8], "units": ["km"]})
d.add({"i": ["san-diego"], "j": ["topeka"], "values": [1.4], "units": ["km"]})

Every time you add data, though, **all** columns must be present!

In [ ]:
# cost per case per 1000 miles

# TODO we could really use a units.get_or_create() function!
try:
    unit_cost_per_case = platform.units.get("USD/km")
except Unit.NotFound:
    unit_cost_per_case = platform.units.create("USD/km")

f = run.optimization.scalars.create(name="f", value=90, unit=unit_cost_per_case)

### Defining `Variable`s and `Equation`s in the scenario

The levels and marginals of these `Variable`s and `Equation`s will be imported to the scenario when reading the model solution.

In [ ]:
# initialize the decision variables and equations
z = run.optimization.variables.create("z")
x = run.optimization.variables.create("x", constrained_to_indexsets=["i", "j"])
supply = run.optimization.equations.create("supply", constrained_to_indexsets=["i"])
demand = run.optimization.equations.create("demand", constrained_to_indexsets=["j"])

### Solve the scenario

In this tutorial, we solve the tutorial using the ``highs`` solver in linopy. 

The ``create_dantzig_model()`` function is a convenience shortcut for setting up a linopy model correctly for the dantzig scenario. Please see ``linopy_model.py`` for details.

The solution data are stored with the model object automatically. ``store_dantzig_solution()`` then stores them in the ixmp4 objects.

In [ ]:
from tutorial.transport.dantzig_model_linopy import (
    create_dantzig_model,
    read_dantzig_solution,
)

linopy_model = create_dantzig_model(run=run)
linopy_model.solve("highs")

read_dantzig_solution(model=linopy_model, run=run)

### Display and analyze the results

In [ ]:
# Display the objective value of the solution
z.levels

In [ ]:
# Display the quantities transported from canning plants to demand locations
pd.DataFrame(x.data)

In [ ]:
# Display the quantities and marginals (shadow prices) of the demand balance constraints
demand.data

In [ ]:
# Display the quantities and marginals (shadow prices) of the supply balance constraints
supply.data